In [1]:
import pandas as pd
import os
import xlwt

In [2]:
def decode_well(well_id):
    return ord(well_id[0]) - 64, int(well_id[1:])


class WorkBook():
    def __init__(self, name):
        self.name = name
        self.line = 0
        self.book = xlwt.Workbook(encoding="utf-8")
        self.sheet1 = self.book.add_sheet(name)
        
        self.write_line(['ID', 902])
        self.write_line(['Name', '384gl'])
        self.write_line(['Type', 384])
        self.write_line(['Category', 'Gel Plate with Lid'])
        self.write_line(['c', 'r', 'Gene'])

    def write_line(self, line_list):
        for entry, column in zip(line_list, range(len(line_list))):
            self.sheet1.write(self.line, column, entry)
        self.line += 1
    
    def write_destination_line(self, gname):
        cp = (self.line - 5) % 24 + 1
        rp = (self.line - 5) // 24 + 1
        self.write_line([cp, rp, gname])
        
    def saveit(self, location=''):
        self.book.save("{}{}.xls".format(location, self.name))

class Summary_file():
    def __init__(self, name, location=''):
        self.w_file = open(location + name +'.tsv','w')
        print('Source Plate\tSource Row\tSource Column\tDestination Plate'
              '\tDestination Row\tDestination Column\tGene', file=self.w_file)
    
    def writeline(self, source_loc, gene, dest_plate, dest_line):
        print(source_loc[0],source_loc[1],source_loc[2], dest_plate, 
              (dest_line - 5) // 24 + 1, (dest_line - 5) % 24 + 1, gene, 
              sep='\t', file=self.w_file)
    
    def closeit(self):
        self.w_file.close()

## SELECT UNIQUE IDs

In [3]:
def select_uniques2(in_table, dom_per=25, sec_per=5, dom_count=20, ret_table=False):
    selection = in_table[(in_table[" % Domint ORF"] >= dom_per) & (in_table[" % 2nd Domint ORF"] <= sec_per) & 
                        (in_table["Domint ORF count"] >= dom_count)]
    if ret_table:
        return selection
    return len(set(selection["  Domint ORF"]))

my_data_db = pd.DataFrame.from_csv('./input/finalYeastOrfeomeDBSummaryFile.txt', sep='\t')

# Formatting
unique_ids = set([i[:i.index('_')] for i in select_uniques2(my_data_db, dom_per=30, ret_table=True)['  Domint ORF']])
print("We have {} Unique IDs that we will ommit.".format(len(unique_ids)))

We have 3041 Unique IDs that we will ommit.


## Make Source Files

In [4]:
my_data = pd.read_excel('./input/Yeast ORFeome HIP ORFeome collection_v3 Details inc 384-format.xlsx')
my_data = my_data.dropna(subset = ['ORF_NAME'])

In [5]:
vals = [(name, decode_well(well)[0], decode_well(well)[1],  poll[-2:]) for well, poll, name in
        zip(list(my_data['scORFeome_384-format_HIP Well']), my_data['scORFeome_384-format_HIP Plate'], my_data['ORF_NAME'])]

locations = {name: (int(plate), rn, cn) for name, rn, cn, plate in vals}
print('Total number of ORFs:', len(my_data))
print('Number of Unique ORFs:', len(locations))

locations = {b:a for a,b in locations.items()}

Total number of ORFs: 5348
Number of Unique ORFs: 5305


In [6]:
empty_colony_count = 1
for plate in range(1, 16):
    source_wb = WorkBook('SPlate{}'.format(plate))
    for row in range(1, 17):
        for column in range(1, 25):
            if (plate, row, column) in locations:
                source_wb.write_line([column, row, locations[(plate, row, column)]])
            else:
                source_wb.write_line([column, row, 'empty_colony_{}'.format(empty_colony_count)])
                empty_colony_count += 1
    source_wb.saveit('./output/')

# Make Destination Files

In [7]:
total_picks = 0
wb_count = 1
picking_order = sorted(locations.keys())

dest_wb = WorkBook('DB_DPlate{}'.format(wb_count))
sum_wfile = Summary_file('Transfer_summary_DB', './output/')
for pick in picking_order:
    
    if locations[pick] not in unique_ids:
        total_picks += 1
        dest_wb.write_destination_line(locations[pick])
        sum_wfile.writeline(pick, locations[pick], wb_count, dest_wb.line - 1)
        if dest_wb.line == 389:
            dest_wb.saveit('./output/')
            wb_count += 1
            dest_wb = WorkBook('DB_DPlate{}'.format(wb_count))

dest_wb.saveit('./output/')
sum_wfile.closeit()
print('Total picks done:', total_picks)

Total picks done: 2264
